# Smart MCP Client with Azure OpenAI and Azure PostgreSQL Flexible server

This notebook demonstrates the **Smart Context-Aware MCP Client** that intelligently handles conversation history in PostgreSQL to maximize LLM token savings while ensuring accurate responses using **Azure OpenAI embeddings** with **DiskANN vector indexing**.

## 🧠 Smart Features:
- **🎯 Context-Aware Matching**: Understands that "my database" = "advworks" or smilar questions in context
- **🔍 Intent Analysis**: Distinguishes schema requests from simple table lists
- **⚡ Dynamic Thresholds**: Adjusts similarity requirements (60%/70%/80%) based on context/intent
- **💾 PostgreSQL Storage**: Stores all conversations with vector similarity search
- **💰 Token Optimization**: Maximum cache hits while preventing false positives
- **🧠 Azure OpenAI Embeddings**: Uses `text-embedding-3-small` (1536 dimensions)
- **⚡ DiskANN Vector Index**: Ultra-high performance vector similarity search
- **📊 Transparent Debugging**: Clear output shows matching logic and decisions


In [ ]:
import os
import warnings
import logging

# Set environment variable to suppress google-crc32c warnings
os.environ['GOOGLE_CRC32C_IGNORE_IMPORT_ERROR'] = '1'

# Suppress warnings before any imports
warnings.filterwarnings("ignore", category=ResourceWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Remove INFO:httpx from output
logging.getLogger("httpx").setLevel(logging.WARNING)
print("✅ Environment configured successfully")

In [ ]:
# Initialize the Smart Context-Aware MCP Client
import importlib
import smart_mcp_client_improved
importlib.reload(smart_mcp_client_improved)

from smart_mcp_client_improved import SmartMCPClientImproved

# Create smart client with context-aware similarity matching
# The 70% threshold is used as base, but dynamic thresholds are applied based on:
# - 60% for context-normalized questions (e.g., "my database" = "advworks")  
# - 70% for regular compatible intent questions
# - 80% for incompatible intent questions (prevents false matches)
# MCP URL is now read from environment variable MCP_URL
smart_client = SmartMCPClientImproved(similarity_threshold=0.70)

# Initialize the smart client
await smart_client.init_chat()

In [ ]:
# Test 1: Ask a simple question about tables (creates baseline cached response)
await smart_client.ask_question("What tables are in the advworks database?")

In [ ]:
# Test 2: Ask for detailed schema - should NOT match simple table list (different intent)
await smart_client.ask_question("Show me the schema for 2-3 tables in advwork with column types")

In [ ]:
# Test 3: Ask similar schema question - should match previous schema response (same intent)
await smart_client.ask_question("What are the table schemas with column data types in advworks?")

In [ ]:
# Test 4: Ask another simple table question - should match first simple response
await smart_client.ask_question("List all tables in advworks")

In [ ]:
# Test 5: Test context-aware matching - "my database" should match "advworks database"
await smart_client.ask_question("What tables are in my database?")

In [ ]:
await smart_client.ask_question("How many employees do I have?")

In [ ]:
await smart_client.ask_question("How many employees are hired?")

In [ ]:
await smart_client.ask_question("What are the curent postgres flex server specs?")

## 🗂️ Session Management

Manage your conversation sessions:

In [ ]:
# Clear current session and start a new one
print("🧹 Clearing current session...")
await smart_client.clear_chat_history()
print("✅ New session started")

In [ ]:
# Test cross-session similarity - should find cached responses from previous sessions
await smart_client.ask_question("What tables are in my db?")

## 🧽 Cleanup

When you're done, properly disconnect from the MCP server:

In [ ]:
# Cleanup and disconnect
await smart_client.cleanup()